In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pymc as pm
import arviz as az
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

In [10]:
# pip install pandas numpy scikit-learn pymc arviz scipy matplotlib seaborn openpyxl

In [9]:
# pip install --upgrade pip

In [11]:
# Load dataset (assuming it's the full Excel file)
data = pd.read_excel("Dataset Price optimization RL.xlsx")

# Rename columns for consistency
data.columns = ['DateTime', 'Date', 'ProductID', 'Cost', 'Price', 'MarginPercent', 'Brand',
                'Category', 'VisitorReturning', 'VisitorSource', 'VisitorOS', 'BasketCount',
                'PurchaseCount', 'Margin', 'Reward']

# Convert DateTime and Date to datetime for consistency
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Date'] = pd.to_datetime(data['Date'])

# Check data
print(data.head())
print(data.info())

# Define features and target
categorical_features = ['Brand', 'Category', 'VisitorSource', 'VisitorOS', 'VisitorReturning']
numerical_features = ['Price', 'MarginPercent']
target = 'PurchaseCount'  # Use PurchaseCount > 0 as binary target for simplicity

# Create binary target (1 if purchased, 0 if not)
data['Purchased'] = (data['PurchaseCount'] > 0).astype(int)

# Handle missing values (if any)
data = data.dropna()

# Aggregate data by ProductID to get average behavior (optional for sparse data)
product_data = data.groupby('ProductID').agg({
    'Price': 'mean', 'MarginPercent': 'mean', 'Purchased': 'mean', 'BasketCount': 'mean',
    'PurchaseCount': 'mean', 'Brand': 'first', 'Category': 'first', 'VisitorSource': 'first',
    'VisitorOS': 'first', 'VisitorReturning': 'mean'
}).reset_index()

# Split features and target
X = product_data[categorical_features + numerical_features]
y = product_data['Purchased']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline with handle_unknown='ignore' to avoid errors with unseen categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'),
         categorical_features)
    ])

# Fit and transform
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
feature_names = (numerical_features +
                 preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist())

# Print shapes for verification
print("Training data shape:", X_train_processed.shape)
print("Test data shape:", X_test_processed.shape)

             DateTime       Date ProductID   Cost   Price  MarginPercent  \
0 2019-09-04 19:03:51 2019-09-04    459238  67.73  309.99       0.321849   
1 2019-09-04 19:06:46 2019-09-04    350105   0.98    3.96       0.292929   
2 2019-09-04 19:06:57 2019-09-04    350275   0.49    2.29       0.327511   
3 2019-09-04 19:07:49 2019-09-04  467960_1  10.75   42.99       0.290300   
4 2019-09-04 19:08:01 2019-09-04    418550   3.12   14.99       0.332221   

          Brand                  Category  VisitorReturning VisitorSource  \
0  Wisch-Star¬Æ          Hygienebereiche                  0        Google   
1       Antikal          Hygienebereiche                  0        Google   
2  Dr. Beckmann          Hygienebereiche                  0        Anders   
3  Wisch-Star¬Æ          Reinigungswagen                  1        Anders   
4  Wisch-Star¬Æ  Wischmop & Bodenwischer                  0        Google   

  VisitorOS  BasketCount  PurchaseCount  Margin  Reward  
0   Android           

/home/ubuntu/.local/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [12]:
import pymc as pm
import arviz as az
import numpy as np

# Convert to numpy arrays if not already done
X_train_np = np.array(X_train_processed)
X_test_np = np.array(X_test_processed)
y_train_np = np.array(y_train)

# Bayesian model with hierarchical grouping by Brand and Category
with pm.Model() as model:
    # Get indices for hierarchical grouping (using original X_train for Brand and Category)
    brand_idx = X_train['Brand'].astype('category').cat.codes.values
    category_idx = X_train['Category'].astype('category').cat.codes.values
    n_brands = len(np.unique(brand_idx))
    n_categories = len(np.unique(category_idx))

    # Priors
    intercept = pm.Normal('intercept', mu=0, sigma=10)
    beta_price = pm.Normal('beta_price', mu=0, sigma=10)
    beta_margin = pm.Normal('beta_margin', mu=0, sigma=10)

    # Hierarchical effects
    brand_mu = pm.Normal('brand_mu', mu=0, sigma=1, shape=n_brands)
    category_mu = pm.Normal('category_mu', mu=0, sigma=1, shape=n_categories)

    # Coefficients for other categorical features (after Price and MarginPercent)
    n_other_features = X_train_np.shape[1] - 2  # Subtract Price and MarginPercent
    beta_other = pm.Normal('beta_other', mu=0, sigma=10, shape=n_other_features)

    # Linear combination
    logits = (intercept +
              beta_price * X_train_np[:, feature_names.index('Price')] +
              beta_margin * X_train_np[:, feature_names.index('MarginPercent')] +
              brand_mu[brand_idx] + category_mu[category_idx] +
              pm.math.dot(X_train_np[:, 2:], beta_other))

    # Likelihood
    likelihood = pm.Bernoulli('likelihood', logit_p=logits, observed=y_train_np)

    # Sample from posterior
    trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42)

# Summarize results
print(az.summary(trace, hdi_prob=0.95))
az.plot_trace(trace)
plt.show()

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-pac


You can find the C code in this temporary file: /tmp/pytensor_compilation_error_g4bb1sf7


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-pac


You can find the C code in this temporary file: /tmp/pytensor_compilation_error_1x1un0br


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-pa

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Squeeze{axis=0}([11])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-pac


You can find the C code in this temporary file: /tmp/pytensor_compilation_error_a6nbwx0g


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Squeeze{axis=0}([700])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.local/lib/python3.12/site-pa


You can find the C code in this temporary file: /tmp/pytensor_compilation_error_7w7rhmrk

You can find the C code in this temporary file: /tmp/pytensor_compilation_error_og31z0vl


CompileError: Compilation failed (return status=1):
/usr/bin/g++ -shared -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -march=znver3 -mmmx -mpopcnt -msse -msse2 -msse3 -mssse3 -msse4.1 -msse4.2 -mavx -mavx2 -msse4a -mno-fma4 -mno-xop -mfma -mno-avx512f -mbmi -mbmi2 -maes -mpclmul -mno-avx512vl -mno-avx512bw -mno-avx512dq -mno-avx512cd -mno-avx512er -mno-avx512pf -mno-avx512vbmi -mno-avx512ifma -mno-avx5124vnniw -mno-avx5124fmaps -mno-avx512vpopcntdq -mno-avx512vbmi2 -mno-gfni -mvpclmulqdq -mno-avx512vnni -mno-avx512bitalg -mno-avx512bf16 -mno-avx512vp2intersect -mno-3dnow -madx -mabm -mno-cldemote -mclflushopt -mclwb -mclzero -mcx16 -mno-enqcmd -mf16c -mfsgsbase -mfxsr -mno-hle -msahf -mno-lwp -mlzcnt -mmovbe -mno-movdir64b -mno-movdiri -mno-mwaitx -mno-pconfig -mno-pku -mno-prefetchwt1 -mprfchw -mno-ptwrite -mrdpid -mrdrnd -mrdseed -mno-rtm -mno-serialize -mno-sgx -msha -mshstk -mno-tbm -mno-tsxldtrk -mvaes -mno-waitpkg -mno-wbnoinvd -mxsav